## Import all the libraries 

In [1]:
from pymongo import MongoClient
import re
from pymongo import ASCENDING, DESCENDING
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize
import requests



## Import the dataset Companies

In [2]:
dbName = "datamad0320"
mongodbURL = f"mongodb://localhost/{dbName}"
print(mongodbURL)
client = MongoClient(mongodbURL, connectTimeoutMS=2000,serverSelectionTimeoutMS=2000)
db = client.get_database()

mongodb://localhost/datamad0320


# First Analysis: Decide the City


## What is the city with more Gaming companies?

This could be interresting to see where to install the new office of our Gaming company.

## Get all the offices with category Gaming

In [3]:
a = list(db.companies.aggregate([{"$unwind":"$offices"},{"$project":{"offices":1,"name":1,"category_code":1}}]))

In [4]:
df = pd.DataFrame(json_normalize(a))
print(df.shape)
df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """Entry point for launching an IPython kernel.


(16705, 12)


,_id,name,category_code,offices.description,offices.address1,offices.address2,offices.zip_code,offices.city,offices.state_code,offices.country_code,offices.latitude,offices.longitude
0,52cdef7c4bab8bd675297d8a,Wetpaint,web,,710 - 2nd Avenue,Suite 1100,98104,Seattle,WA,USA,47.603122,-122.333253
1,52cdef7c4bab8bd675297d8a,Wetpaint,web,,270 Lafayette Street,Suite 505,10012,New York,NY,USA,40.723731,-73.996431
2,52cdef7c4bab8bd675297d8b,AdventNet,enterprise,Headquarters,4900 Hopyard Rd.,Suite 310,94588,Pleasanton,CA,USA,37.692934,-121.904945
3,52cdef7c4bab8bd675297d8c,Zoho,software,Headquarters,4900 Hopyard Rd,Suite 310,94588,Pleasanton,CA,USA,37.692934,-121.904945
4,52cdef7c4bab8bd675297d8f,Omnidrive,network_hosting,,Suite 200,654 High Street,94301,Palo Alto,CA,ISR,NaN,NaN


## Filter only Category Gaming that has the city information

In [5]:
df_gaming = df

df_gaming.drop(df_gaming[df_gaming.category_code != "games_video"].index, axis=0, inplace=True)


In [6]:
df_gaming.dropna(subset = ['offices.city'])
df_gaming.shape

(956, 12)

## Result: New York

In [7]:
df_gaming["offices.city"].value_counts()

New York           75
San Francisco      68
                   54
London             36
Los Angeles        31
                   ..
Singapore           1
Malvern             1
Shelbyville         1
Mississauga, ON     1
Belfast             1
Name: offices.city, Length: 363, dtype: int64

## Designers requirement: City with design companies

Search for cities with design companies, because this will be one of our requirements for the new office. 

In [8]:
df = pd.DataFrame(json_normalize(a))
print(df.shape)
df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """Entry point for launching an IPython kernel.


(16705, 12)


,_id,name,category_code,offices.description,offices.address1,offices.address2,offices.zip_code,offices.city,offices.state_code,offices.country_code,offices.latitude,offices.longitude
0,52cdef7c4bab8bd675297d8a,Wetpaint,web,,710 - 2nd Avenue,Suite 1100,98104,Seattle,WA,USA,47.603122,-122.333253
1,52cdef7c4bab8bd675297d8a,Wetpaint,web,,270 Lafayette Street,Suite 505,10012,New York,NY,USA,40.723731,-73.996431
2,52cdef7c4bab8bd675297d8b,AdventNet,enterprise,Headquarters,4900 Hopyard Rd.,Suite 310,94588,Pleasanton,CA,USA,37.692934,-121.904945
3,52cdef7c4bab8bd675297d8c,Zoho,software,Headquarters,4900 Hopyard Rd,Suite 310,94588,Pleasanton,CA,USA,37.692934,-121.904945
4,52cdef7c4bab8bd675297d8f,Omnidrive,network_hosting,,Suite 200,654 High Street,94301,Palo Alto,CA,ISR,NaN,NaN


## Filter only Category Design/Web that has the city information

In [9]:
df_design = df

df_design.drop(df_design[(df_design['category_code'] != "web") & (df_design['category_code'] != "design")].index , inplace=True)

In [10]:
print(df_design.shape)
df_design.head()

(3085, 12)


,_id,name,category_code,offices.description,offices.address1,offices.address2,offices.zip_code,offices.city,offices.state_code,offices.country_code,offices.latitude,offices.longitude
0,52cdef7c4bab8bd675297d8a,Wetpaint,web,,710 - 2nd Avenue,Suite 1100,98104,Seattle,WA,USA,47.603122,-122.333253
1,52cdef7c4bab8bd675297d8a,Wetpaint,web,,270 Lafayette Street,Suite 505,10012,New York,NY,USA,40.723731,-73.996431
5,52cdef7c4bab8bd675297d90,Postini,web,None,"959 Skyway Road, Suite 200",None,94070,San Carlos,CA,USA,37.506885,-122.247573
6,52cdef7c4bab8bd675297d91,Geni,web,Headquarters,9229 W. Sunset Blvd.,,90069,West Hollywood,CA,USA,34.090368,-118.393064
7,52cdef7c4bab8bd675297d96,Gizmoz,web,None,None,None,None,Menlo Park,CA,USA,37.484130,-122.169472


In [11]:
df_design.dropna(subset = ['offices.city'])
df_design.shape

(3085, 12)

## Result: New York is the second city with more design companies

In [12]:
df_design["offices.city"].value_counts()

San Francisco                   217
New York                        193
                                184
London                          113
Seattle                          63
                               ... 
Guangzhou                         1
Lone Butte ,British Columbia      1
Acton                             1
Naliya                            1
Pompano Beach                     1
Name: offices.city, Length: 900, dtype: int64

Now that we already have New York as our city, due to their number of gaming and design companies, we need to continue to investigate in which coordinate we will launch the new office.

# Requirement: Nearby Design companies

Designers like to go to design talks and share knowledge. There must be some nearby companies that also do design.

Designers represents 23% of the total employees of the company.

Filter all the DESIGN COMPANIES COORDINATES IN NEW YORK. Our new office needs to be nearby some of these coordinates.

In [13]:
ny_offices = list(db.companies.find({"offices.city":{"$eq":"New York"}},{"offices":1,"name":1,"category_code":1}))


In [14]:
data = pd.DataFrame(ny_offices) 
data.shape

(832, 4)

In [15]:
data = data.explode("offices")
data.head()

,_id,name,category_code,offices
0,52cdef7c4bab8bd675297d8a,Wetpaint,web,"{'description': '', 'address1': '710 - 2nd Ave..."
0,52cdef7c4bab8bd675297d8a,Wetpaint,web,"{'description': '', 'address1': '270 Lafayette..."
1,52cdef7c4bab8bd675297d9d,Joost,games_video,"{'description': '', 'address1': '100 5th Ave F..."
2,52cdef7c4bab8bd675297d9f,Viacom,web,"{'description': 'HQ', 'address1': '1515 Broadw..."
3,52cdef7c4bab8bd675297da7,AddThis,advertising,"{'description': 'HQ - Virginia', 'address1': '..."


In [16]:
def officeToGeoPoint(row):
    office = row.offices
    if type(office) == dict:
        if 'latitude' in office and 'longitude' in office:
            if(type(office["latitude"])) == float and type(office["longitude"]) == float:
                return ({
                    "type":"Point",
                    "coordinates":[office["longitude"],office["latitude"]]
                },"success")
            else:
                return(None,"Invalid lat lat and long")
        else:
            return (None,"No lat and long keys in office dict")
    return (None,"No office")

In [17]:
cleaned_data = data.apply(officeToGeoPoint,axis=1, result_type="expand")
cleaned_data.columns = ["office","clean_state"]

cleaned_data.head()

,office,clean_state
0,"{'type': 'Point', 'coordinates': [-122.333253,...",success
0,"{'type': 'Point', 'coordinates': [-73.9964312,...",success
1,"{'type': 'Point', 'coordinates': [-74.0094471,...",success
2,"{'type': 'Point', 'coordinates': [-73.986011, ...",success
3,"{'type': 'Point', 'coordinates': [-77.245195, ...",success


In [18]:
data_processed = pd.concat([data,cleaned_data], axis=1)

In [19]:
data_processed = data_processed[["name","category_code","office","clean_state"]]

## Filter only companies with coordinates

In [20]:
data_processed = data_processed[data_processed.clean_state == 'success']
data_processed.head()

,name,category_code,office,clean_state
0,Wetpaint,web,"{'type': 'Point', 'coordinates': [-122.333253,...",success
0,Wetpaint,web,"{'type': 'Point', 'coordinates': [-73.9964312,...",success
1,Joost,games_video,"{'type': 'Point', 'coordinates': [-74.0094471,...",success
2,Viacom,web,"{'type': 'Point', 'coordinates': [-73.986011, ...",success
3,AddThis,advertising,"{'type': 'Point', 'coordinates': [-77.245195, ...",success


## Filter only Design/Web companies

In [21]:
data_final = data_processed[data_processed.category_code == 'web']
data_final.head()

,name,category_code,office,clean_state
0,Wetpaint,web,"{'type': 'Point', 'coordinates': [-122.333253,...",success
0,Wetpaint,web,"{'type': 'Point', 'coordinates': [-73.9964312,...",success
2,Viacom,web,"{'type': 'Point', 'coordinates': [-73.986011, ...",success
5,Meetup,web,"{'type': 'Point', 'coordinates': [-73.995722, ...",success
12,Meebo,web,"{'type': 'Point', 'coordinates': [-122.079017,...",success


In [22]:
data_final.shape

(179, 4)

We have a range of 179 coordinates in New York to use as a reference to search for the new office. 

## Convert into Json

In [23]:
data_final.to_json("../Input/data_final.json",orient="records")

## Get the point of reference of data_final:

In [24]:
data_mean = list(db.data_prepared.aggregate([{"$unwind":"$office"},{"$project":{"office":1,"name":1,"category_code":1}}]))

In [25]:
data_mean = pd.DataFrame(json_normalize(data_mean))
data_mean

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """Entry point for launching an IPython kernel.


,_id,name,category_code,office.type,office.coordinates
0,5e9c487168b89a1cb3809e1e,Meetup,web,Point,"[-73.995722, 40.72604]"
1,5e9c487168b89a1cb3809e1f,Meebo,web,Point,"[-122.079017, 37.393679]"
2,5e9c487168b89a1cb3809e20,Buzzmetrics,web,Point,"[-73.991933, 40.731118]"
3,5e9c487168b89a1cb3809e21,Heavy,web,Point,"[-73.98962, 40.753385]"
4,5e9c487168b89a1cb3809e22,Viacom,web,Point,"[-73.986011, 40.757725]"
...,...,...,...,...,...
174,5e9c487168b89a1cb3809ecc,Strategist News,web,Point,"[-73.9918181, 40.7489381]"
175,5e9c487168b89a1cb3809ecd,Zemoga,web,Point,"[-73.989947, 40.744697]"
176,5e9c487168b89a1cb3809ece,Zemoga,web,Point,"[-73.4313029, 41.1932851]"
177,5e9c487168b89a1cb3809ecf,ChallengePost,web,Point,"[-74.00717, 40.7408042]"


In [26]:
data_mean.describe()

TypeError: unhashable type: 'list'

Exception ignored in: 'pandas._libs.index.IndexEngine._call_map_locations'
Traceback (most recent call last):
  File "pandas/_libs/hashtable_class_helper.pxi", line 1653, in pandas._libs.hashtable.PyObjectHashTable.map_locations
TypeError: unhashable type: 'list'


,_id,name,category_code,office.type,office.coordinates
count,179,179,179,179,179
unique,179,154,1,1,151
top,5e9c487168b89a1cb3809e1e,Allopass,web,Point,"[-73.985506, 40.757929]"
freq,1,4,179,179,6


## Point of reference: Our Coordinates

Coordinates: [-73.985506, 40.757929]

Address: 159 West 45th Street, New York, NY 10036*

*We have searched for the address on the internet though the coordinates.


# Confirming the Requirement: Nearby Design companies

## Geocoding

Gives the coordinates from an address.

In [27]:
def geocode(address):
    res = requests.get(f"https://geocode.xyz/{address}",params={"json":1})
    data = res.json()
    print(res)
    return {
    "type": "Point",
    "coordinates": [float(data["longt"]), float(data["latt"])]
  }

## Query Nearby

Filter the coordinates that are in the requested radius from a specific point of reference.

In [28]:
def getOfficeNear(address, maxDist=1000):
    point = geocode(address)
    return {
       "office": {
         "$near": {
           "$geometry": point,
           "$maxDistance": maxDist,
         }
       } 
    }

## Using our Point of reference

In [29]:
query = getOfficeNear("159 West 45th Street, New York, NY 10036")
query 

<Response [200]>


{'office': {'$near': {'$geometry': {'type': 'Point',
    'coordinates': [-73.98439, 40.75748]},
   '$maxDistance': 1000}}}

## Result:

From our point of reference (-73.985506, 40.757929), we have 32 design companies in a 1KM radius. This fulfill one of the requirements.

In [30]:
cur = db.data_prepared.find(query, {"_id":0})
print(cur.count())
result = list(cur)
result

32


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  


[{'name': 'Tacoda',
  'category_code': 'web',
  'office': {'type': 'Point', 'coordinates': [-73.985506, 40.757929]},
  'clean_state': 'success'},
 {'name': 'Totspot',
  'category_code': 'web',
  'office': {'type': 'Point', 'coordinates': [-73.985506, 40.757929]},
  'clean_state': 'success'},
 {'name': 'Kidmondo',
  'category_code': 'web',
  'office': {'type': 'Point', 'coordinates': [-73.985506, 40.757929]},
  'clean_state': 'success'},
 {'name': 'SnapVillage',
  'category_code': 'web',
  'office': {'type': 'Point', 'coordinates': [-73.985506, 40.757929]},
  'clean_state': 'success'},
 {'name': 'Blog Talk Radio',
  'category_code': 'web',
  'office': {'type': 'Point', 'coordinates': [-73.985506, 40.757929]},
  'clean_state': 'success'},
 {'name': 'Gamers Media',
  'category_code': 'web',
  'office': {'type': 'Point', 'coordinates': [-73.985506, 40.757929]},
  'clean_state': 'success'},
 {'name': 'Viacom',
  'category_code': 'web',
  'office': {'type': 'Point', 'coordinates': [-73.98601

## Get the final coordinates

In [31]:
def easyLatLng(row):
    of = row.office
    return {
        "latitude":of["coordinates"][1],
        "longitude":of["coordinates"][0]
    }

df_final = pd.DataFrame(result)

df_final = pd.concat([df_final, df_final.apply(easyLatLng, axis=1, result_type="expand")], axis=1)
df_final

,name,category_code,office,clean_state,latitude,longitude
0,Tacoda,web,"{'type': 'Point', 'coordinates': [-73.985506, ...",success,40.757929,-73.985506
1,Totspot,web,"{'type': 'Point', 'coordinates': [-73.985506, ...",success,40.757929,-73.985506
2,Kidmondo,web,"{'type': 'Point', 'coordinates': [-73.985506, ...",success,40.757929,-73.985506
3,SnapVillage,web,"{'type': 'Point', 'coordinates': [-73.985506, ...",success,40.757929,-73.985506
4,Blog Talk Radio,web,"{'type': 'Point', 'coordinates': [-73.985506, ...",success,40.757929,-73.985506
5,Gamers Media,web,"{'type': 'Point', 'coordinates': [-73.985506, ...",success,40.757929,-73.985506
6,Viacom,web,"{'type': 'Point', 'coordinates': [-73.986011, ...",success,40.757725,-73.986011
7,Giant Realm,web,"{'type': 'Point', 'coordinates': [-73.9827553,...",success,40.756666,-73.982755
8,Livekick,web,"{'type': 'Point', 'coordinates': [-73.986951, ...",success,40.756054,-73.986951
9,Healism,web,"{'type': 'Point', 'coordinates': [-73.986951, ...",success,40.756054,-73.986951


In [32]:
df_final = df_final.drop(columns=['name', 'category_code', 'office', 'clean_state'])
df_final["name"] = "Design companies"
df_final.head()

,latitude,longitude,name
0,40.757929,-73.985506,Design companies
1,40.757929,-73.985506,Design companies
2,40.757929,-73.985506,Design companies
3,40.757929,-73.985506,Design companies
4,40.757929,-73.985506,Design companies


# Requirement: Near the Airport

Account managers need to travel a lot.

Let's check the distance from our coordinates to the airport and see if is not so far away.

In [33]:
def getAirportNear(address, maxDist=10000):
    point = geocode(address)
    return {
       "office": {
         "$near": {
           "$geometry": point,
           "$maxDistance": maxDist,
         }
       } 
    }

In [35]:
query_airport = getAirportNear("LaGuardia Airport Entrance, Queens, NY 11371, Estados Unidos")
query_airport 

<Response [200]>


{'office': {'$near': {'$geometry': {'type': 'Point',
    'coordinates': [-73.89209, 40.73867]},
   '$maxDistance': 10000}}}

In [36]:
cur = db.data_prepared.find(query_airport, {"_id":0})
print(cur.count())
result_airport = list(cur)
result_airport

131


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  


[{'name': 'Sonibyte',
  'category_code': 'web',
  'office': {'type': 'Point', 'coordinates': [-73.950845, 40.769466]},
  'clean_state': 'success'},
 {'name': 'Disrupto',
  'category_code': 'web',
  'office': {'type': 'Point', 'coordinates': [-73.9565551, 40.714279]},
  'clean_state': 'success'},
 {'name': 'CarZen',
  'category_code': 'web',
  'office': {'type': 'Point', 'coordinates': [-73.951865, 40.76945]},
  'clean_state': 'success'},
 {'name': 'WebVet',
  'category_code': 'web',
  'office': {'type': 'Point', 'coordinates': [-73.9513502, 40.7717776]},
  'clean_state': 'success'},
 {'name': 'Overtime Media',
  'category_code': 'web',
  'office': {'type': 'Point', 'coordinates': [-73.953502, 40.779947]},
  'clean_state': 'success'},
 {'name': 'Minyanville',
  'category_code': 'web',
  'office': {'type': 'Point', 'coordinates': [-73.971509, 40.755635]},
  'clean_state': 'success'},
 {'name': 'Broadband Reports',
  'category_code': 'web',
  'office': {'type': 'Point', 'coordinates': [-7

In [39]:
def easyLatLng(row):
    of = row.office
    return {
        "latitude":of["coordinates"][1],
        "longitude":of["coordinates"][0]
    }

df_airport = pd.DataFrame(result_airport)

df_aiport = pd.concat([df_airport, df_airport.apply(easyLatLng, axis=1, result_type="expand")], axis=1)


Filter only our Coordinates: [-73.985506, 40.757929]

In [40]:
df_aiport = df_aiport[df_aiport.latitude == 40.757929]
df_aiport

,name,category_code,office,clean_state,latitude,longitude
44,Totspot,web,"{'type': 'Point', 'coordinates': [-73.985506, ...",success,40.757929,-73.985506
45,Tacoda,web,"{'type': 'Point', 'coordinates': [-73.985506, ...",success,40.757929,-73.985506
46,Kidmondo,web,"{'type': 'Point', 'coordinates': [-73.985506, ...",success,40.757929,-73.985506
47,Gamers Media,web,"{'type': 'Point', 'coordinates': [-73.985506, ...",success,40.757929,-73.985506
48,Blog Talk Radio,web,"{'type': 'Point', 'coordinates': [-73.985506, ...",success,40.757929,-73.985506
49,SnapVillage,web,"{'type': 'Point', 'coordinates': [-73.985506, ...",success,40.757929,-73.985506


## Result: 

Our coordinates [-73.985506, 40.757929] is less than 10KM away from the Airport La Guardia, which means that the executives won't take too long to get there and go on their business trips. 

# Concat all the coordinates into one dataframe to export in Tableau

In [ ]:
hjsnfdldlm

In [ ]:
df_tableau = pd.concat([df_requirements,df_final], axis=0)